In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow  as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

from tensorflow.keras.callbacks import *

from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

import os
import datetime

In [132]:
frame = pd.read_csv(r'20172018/ausleihe_8AveW31St.csv')
#frame = pd.read_csv(r'ausleihen_mit_wetter.csv')
df = pd.DataFrame(frame)
df.head()

,triptime,start,end,stationid,station1,lat1,long1,endid,endname,lan2,long2,bikeid,usertype,birthyear,gender
0,2325,2017-01-01 00:13:34,2017-01-01 00:52:19,3255.0,8 Ave & W 31 St,40.750585,-73.994685,532.0,S 5 Pl & S 4 St,40.710451,-73.960876,17252,Subscriber,1980.0,2
1,672,2017-01-01 00:33:18,2017-01-01 00:44:31,3255.0,8 Ave & W 31 St,40.750585,-73.994685,167.0,E 39 St & 3 Ave,40.748901,-73.976049,26341,Subscriber,1966.0,1
2,590,2017-01-01 01:10:05,2017-01-01 01:19:56,3255.0,8 Ave & W 31 St,40.750585,-73.994685,345.0,W 13 St & 6 Ave,40.736494,-73.997044,20032,Subscriber,1981.0,1
3,1052,2017-01-01 01:11:48,2017-01-01 01:29:20,3255.0,8 Ave & W 31 St,40.750585,-73.994685,151.0,Cleveland Pl & Spring St,40.722104,-73.997249,27041,Subscriber,1990.0,1
4,231,2017-01-01 04:54:58,2017-01-01 04:58:50,3255.0,8 Ave & W 31 St,40.750585,-73.994685,509.0,9 Ave & W 22 St,40.745497,-74.001971,24916,Subscriber,1967.0,1


In [4]:
df.isnull().sum()

triptime        0
start           0
end             0
stationid       0
station1        0
lat1            0
long1           0
endid           0
endname         0
lan2            0
long2           0
bikeid          0
usertype       62
birthyear    4926
gender          0
dtype: int64

In [133]:
df = df.drop(columns=['usertype', 'birthyear', 'gender','endname','endid','lan2','long2'])

In [102]:
df.head()

,triptime,start,end,stationid,station1,lat1,long1,bikeid
0,2325,2017-01-01 00:13:34,2017-01-01 00:52:19,3255.0,8 Ave & W 31 St,40.750585,-73.994685,17252
1,672,2017-01-01 00:33:18,2017-01-01 00:44:31,3255.0,8 Ave & W 31 St,40.750585,-73.994685,26341
2,590,2017-01-01 01:10:05,2017-01-01 01:19:56,3255.0,8 Ave & W 31 St,40.750585,-73.994685,20032
3,1052,2017-01-01 01:11:48,2017-01-01 01:29:20,3255.0,8 Ave & W 31 St,40.750585,-73.994685,27041
4,231,2017-01-01 04:54:58,2017-01-01 04:58:50,3255.0,8 Ave & W 31 St,40.750585,-73.994685,24916


In [95]:
df.dtypes

triptime       int64
start         object
end           object
stationid    float64
station1      object
lat1         float64
long1        float64
bikeid         int64
dtype: object

In [134]:
df = df.drop(columns=['end','stationid'])

In [138]:
df['start'] = pd.to_datetime(df['start'], errors='coerce')

In [91]:
# Drop Trips shorter than 90 seconds
df = df.loc[((df["triptime"]>90)),:]
df.head()

,triptime,start,station1,lat1,long1,bikeid
0,2325,2017-01-01 00:13:34,8 Ave & W 31 St,40.750585,-73.994685,17252
1,672,2017-01-01 00:33:18,8 Ave & W 31 St,40.750585,-73.994685,26341
2,590,2017-01-01 01:10:05,8 Ave & W 31 St,40.750585,-73.994685,20032
3,1052,2017-01-01 01:11:48,8 Ave & W 31 St,40.750585,-73.994685,27041
4,231,2017-01-01 04:54:58,8 Ave & W 31 St,40.750585,-73.994685,24916


In [105]:
df['dayofweek'] = df['start'].dt.day_of_week
df.head()

,triptime,start,station1,lat1,long1,bikeid,dayofweek
0,2325,2017-01-01 00:13:34,8 Ave & W 31 St,40.750585,-73.994685,17252,6
1,672,2017-01-01 00:33:18,8 Ave & W 31 St,40.750585,-73.994685,26341,6
2,590,2017-01-01 01:10:05,8 Ave & W 31 St,40.750585,-73.994685,20032,6
3,1052,2017-01-01 01:11:48,8 Ave & W 31 St,40.750585,-73.994685,27041,6
4,231,2017-01-01 04:54:58,8 Ave & W 31 St,40.750585,-73.994685,24916,6


In [106]:
df['hour'] = df['start'].dt.hour
df.head()

,triptime,start,station1,lat1,long1,bikeid,dayofweek,hour
0,2325,2017-01-01 00:13:34,8 Ave & W 31 St,40.750585,-73.994685,17252,6,0
1,672,2017-01-01 00:33:18,8 Ave & W 31 St,40.750585,-73.994685,26341,6,0
2,590,2017-01-01 01:10:05,8 Ave & W 31 St,40.750585,-73.994685,20032,6,1
3,1052,2017-01-01 01:11:48,8 Ave & W 31 St,40.750585,-73.994685,27041,6,1
4,231,2017-01-01 04:54:58,8 Ave & W 31 St,40.750585,-73.994685,24916,6,4


In [108]:
df = df[df.dayofweek != 6]

In [136]:
df

,triptime,start,station1,lat1,long1,bikeid
0,2325,2017-01-01 00:13:34.000,8 Ave & W 31 St,40.750585,-73.994685,17252
1,672,2017-01-01 00:33:18.000,8 Ave & W 31 St,40.750585,-73.994685,26341
2,590,2017-01-01 01:10:05.000,8 Ave & W 31 St,40.750585,-73.994685,20032
3,1052,2017-01-01 01:11:48.000,8 Ave & W 31 St,40.750585,-73.994685,27041
4,231,2017-01-01 04:54:58.000,8 Ave & W 31 St,40.750585,-73.994685,24916
...,...,...,...,...,...,...
177175,1418,2018-12-31 20:15:15.944,8 Ave & W 31 St,40.750585,-73.994685,35705
177176,4583,2018-12-31 20:39:27.685,8 Ave & W 31 St,40.750585,-73.994685,34734
177177,1237,2018-12-31 21:21:38.305,8 Ave & W 31 St,40.750585,-73.994685,17880
177178,2636,2018-12-31 21:24:39.581,8 Ave & W 31 St,40.750585,-73.994685,35190


In [80]:
df['start'] = df['start'].dt.date

In [66]:
df_cleaned = df.groupby([df['start'],df['hour'],df['dayofweek']])['bikeid'].agg('count')

In [157]:
# Daten ohne Sachen gedroppt - nur Columns gelöscht

df_cleaned = df.groupby([df['start'].dt.year,df['start'].dt.month,df['start'].dt.day,df['start'].dt.hour])['bikeid'].agg(['count'])

In [158]:
df_cleaned.head()

count
start start start start       
2017  1     1     0          2
                  1          2
                  4          1
                  5          1
                  11         2

In [154]:
df_cleaned.to_csv('2017_2018_ausleihen_count.csv')

In [67]:
df_cleaned = pd.DataFrame(df_cleaned)

In [68]:
df_cleaned.sort_values(by=['start','hour','dayofweek'])

bikeid
start      hour dayofweek        
2017-01-01 0    6               2
           1    6               2
           4    6               1
           5    6               1
           11   6               2
...                           ...
2018-12-31 15   0               3
           16   0               4
           20   0               2
           21   0               2
           22   0               1

[14151 rows x 1 columns]

In [78]:
#extract year of start-date
df['year'] = df['start'].dt.year
df.head()

,triptime,start,stationid,station1,lat1,long1,bikeid,dayofweek,hour,year
0,2325,2017-01-01 00:13:34,3255.0,8 Ave & W 31 St,40.750585,-73.994685,17252,6,0,2017
1,672,2017-01-01 00:33:18,3255.0,8 Ave & W 31 St,40.750585,-73.994685,26341,6,0,2017
2,590,2017-01-01 01:10:05,3255.0,8 Ave & W 31 St,40.750585,-73.994685,20032,6,1,2017
3,1052,2017-01-01 01:11:48,3255.0,8 Ave & W 31 St,40.750585,-73.994685,27041,6,1,2017
4,231,2017-01-01 04:54:58,3255.0,8 Ave & W 31 St,40.750585,-73.994685,24916,6,4,2017


In [79]:
#extract month of start-date
df['month'] = df['start'].dt.month
df.head()

,triptime,start,stationid,station1,lat1,long1,bikeid,dayofweek,hour,year,month
0,2325,2017-01-01 00:13:34,3255.0,8 Ave & W 31 St,40.750585,-73.994685,17252,6,0,2017,1
1,672,2017-01-01 00:33:18,3255.0,8 Ave & W 31 St,40.750585,-73.994685,26341,6,0,2017,1
2,590,2017-01-01 01:10:05,3255.0,8 Ave & W 31 St,40.750585,-73.994685,20032,6,1,2017,1
3,1052,2017-01-01 01:11:48,3255.0,8 Ave & W 31 St,40.750585,-73.994685,27041,6,1,2017,1
4,231,2017-01-01 04:54:58,3255.0,8 Ave & W 31 St,40.750585,-73.994685,24916,6,4,2017,1


In [81]:
df_cleaned = df.groupby([df['start'],df['year'],df['month'],df['dayofweek'],df['hour'],])['bikeid'].agg('count')

In [82]:
df_cleaned = pd.DataFrame(df_cleaned)

In [58]:
df_cleaned.dtypes

bikeid    int64
dtype: object

In [59]:
df_cleaned.sort_values(by=['year','month','dayofweek','hour','start'])

bikeid
year month dayofweek hour start                          
2017 1     0         0    2017-01-30 00:43:13.000       1
                     1    2017-01-23 01:09:53.000       1
                          2017-01-23 01:27:42.000       1
                     2    2017-01-16 02:13:45.000       1
                          2017-01-30 02:04:06.000       1
...                                                   ...
2018 12    6         23   2018-12-09 23:13:01.394       1
                          2018-12-16 23:19:55.347       1
                          2018-12-23 23:01:22.763       1
                          2018-12-30 23:52:19.535       1
                          2018-12-30 23:55:04.777       1

[175028 rows x 1 columns]

In [83]:
df_cleaned.to_csv('2017_2018_ausleihen_count_new.csv')

In [60]:
df_cleaned['start'] = pd.to_datetime(df_cleaned['start']).dt.date

KeyError: 'start'